1. Develop an image classification model based on transformer architecture without relying on pre-implemented transformer or self-attention modules such as torch.nn.Transformer or torch.nn.MultiheadAttention.


In [1]:
from modules.config import ViTConfig, TrainingConfig, DataConfig
from modules.ViT import VisionTransformer

import torch

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet152  # For comparison
from torch.utils.data import DataLoader, Subset


import numpy as np

from dataclasses import asdict
from modules.pipeline import train_and_evaluate_model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Prepare data
data_config = DataConfig.base()

# DEBUG
# data_config.debug = True
# data_config.batch_size = 2

In [4]:
train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandAugment(num_ops=2),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

In [5]:
trainset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
valset = datasets.CIFAR10(root="./data", train=True, download=True, transform=val_transform)

# Calculate split sizes
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size

# Generate indices for splitting
indices = list(range(len(trainset)))
np.random.shuffle(indices)
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Create subset datasets
train_data = Subset(trainset, train_indices)
val_data = Subset(valset, val_indices)
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=val_transform)
classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

if data_config.debug:
    train_data = Subset(train_data, list(range(256)))
    val_data = Subset(val_data, list(range(256)))
    test_data = Subset(test_data, list(range(256)))

train_loader = DataLoader(
    train_data,
    batch_size=data_config.batch_size,
    shuffle=True,
    num_workers=data_config.num_workers,
    pin_memory=data_config.pin_memory,
)
val_loader = DataLoader(
    val_data,
    batch_size=data_config.batch_size,
    num_workers=data_config.num_workers,
    pin_memory=data_config.pin_memory,
)
test_loader = DataLoader(
    test_data,
    batch_size=data_config.batch_size,
    num_workers=data_config.num_workers,
    pin_memory=data_config.pin_memory,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
def compare_models(
    vit_model,
    cnn_model,
    num_classes,
    train_loader,
    val_loader,
    test_loader,
    vit_train_config: dict[str, any],
    cnn_train_config: dict[str, any],
    **kwargs,
):

    print("Evaluating ViT Model...")
    vit_metrics = train_and_evaluate_model(
        vit_model, num_classes, train_loader, val_loader, test_loader, **vit_train_config
    )

    print("Evaluating CNN Model...")
    cnn_metrics = train_and_evaluate_model(
        cnn_model, num_classes, train_loader, val_loader, test_loader, **cnn_train_config
    )
    metrics = [
        "Test Accuracy",
        "Training Time (s)",
        "Model Size",
        "Avg Inference Time (s)",
        "F1 Score",
        "AUROC",
    ]

    metric_keys = [
        "test_accuracy",
        "training_time",
        "model_size",
        "avg_inference_time",
        "test_f1",
        "test_auroc",
    ]

    # Initialize comparison dictionary with metrics
    comparison = {"Metric": metrics}

    # Add ViT and CNN metrics with proper length checking
    for model_name, metrics_dict in [("ViT", vit_metrics), ("CNN", cnn_metrics)]:
        comparison[model_name] = [metrics_dict.get(key, "N/A") for key in metric_keys]

    # Create DataFrame and save to CSV
    import pandas as pd
    import datetime
    import pathlib

    # print(comparison)
    now = datetime.now().srtftime("%Y-%m-%d-%H-%M-%S")
    pathlib.Path(f"results/{now}").mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(comparison)
    df.to_csv(f"results/{now}/model_comparison.csv", index=False)

    # Print comparison table
    from tabulate import tabulate

    print("\nModel Comparison:")
    print(tabulate(df, headers="keys", tablefmt="grid"))

    # Save model predictions
    predictions_df = pd.DataFrame(
        {
            "ViT Predictions": vit_metrics.get("test_predictions", []),
            "CNN Predictions": cnn_metrics.get("test_predictions", []),
            "Targets": vit_metrics.get("test_targets", []),
        }
    )

    predictions_df.to_csv(f"results/{now}/model_predictions.csv", index=False)

    # Save Training Configs for each
    import json

    with open(f"results/{now}/vit_train_config.json", "w") as f:
        json.dump(vit_train_config, f)

    with open(f"results/{now}/cnn_train_config.json", "w") as f:
        json.dump(cnn_train_config, f)

In [7]:
vit_config = ViTConfig.base()
vit_model = VisionTransformer(**asdict(vit_config))
cnn_model = resnet152()

vit_train_config = TrainingConfig.vit_base()
cnn_train_config = TrainingConfig.resnet152()

# DEBUG
# vit_train_config.epochs = 1
# cnn_train_config.epochs = 1

compare_models(
    vit_model,
    cnn_model,
    data_config.num_classes,
    train_loader,
    val_loader,
    test_loader,
    asdict(vit_train_config),
    asdict(cnn_train_config),
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Evaluating ViT Model...


Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | model      | VisionTransformer  | 2.7 M  | train
1 | criterion  | CrossEntropyLoss   | 0      | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
5 | test_f1    | MulticlassF1Score  | 0      | train
6 | test_auroc | MulticlassAUROC    | 0      | train
----------------------------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.790    Total estimated model params size (MB)
121       Modules in train mode
0         Modules in eval mode


Epoch 19: 100%|██████████| 157/157 [00:06<00:00, 24.01it/s, v_num=7, train_loss=1.670, train_acc=0.469, val_loss=1.200, val_acc=0.684]

/home/work/workdir/vit-assignment/.conda/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Evaluating CNN Model...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | model      | ResNet             | 58.2 M | train
1 | criterion  | CrossEntropyLoss   | 0      | train
2 | train_acc  | MulticlassAccuracy | 0      | train
3 | val_acc    | MulticlassAccuracy | 0      | train
4 | test_acc   | MulticlassAccuracy | 0      | train
5 | test_f1    | MulticlassF1Score  | 0      | train
6 | test_auroc | MulticlassAUROC    | 0      | train
----------------------------------------------------------
58.2 M    Trainable params
0         Non-trainable params
58.2 M    Total params
232.657   Total estimated model params size (MB)
429       Modules in train mode
0         Modules in eval mode


Epoch 33: 100%|██████████| 157/157 [00:14<00:00, 10.66it/s, v_num=0, train_loss=2.080, train_acc=0.250, val_loss=7.280, val_acc=0.268]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:03<00:00, 12.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   avg_inference_time      0.021277153864502907
       model_size               58164296.0
        test_acc            0.2685000002384186
       test_auroc            0.728996217250824
         test_f1            0.2315855622291565
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'Metric': ['Test Accuracy', 'Training Time (s)', 'Model Size', 'Avg Inference Time (s)', 'F1 Score', 'AUROC'], 'ViT': [0.833299994468689, 542.2475743638352, 2697610, 0.006124526262283325, 0.8342033624649048, 0.9820541739463806], 'CNN': [0.2685000002384186, 497.98931500688195, 58164298, 0.0